In [1]:
# Basic library
import nltk
import logging

import pandas as pd
import matplotlib.pyplot as plt

from nltk.tokenize.treebank import TreebankWordDetokenizer
from nltk.tokenize.treebank import TreebankWordTokenizer
from nltk.tokenize.util import align_tokens
from nltk.tokenize.api import TokenizerI
from nltk.tokenize import word_tokenize
from recordlinkage.index import Block
import recordlinkage as rl
import recordlinkage
import jellyfish
import warnings
import numpy as np
from sklearn.metrics import confusion_matrix
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


pd.set_option('display.max_columns', 100)

In [3]:
#datasetsDef = ['seed2010_conv.txt', 'seed2011_conv.txt', 'seed2012_conv.txt', 'seed2013_conv.txt', 'seed2014_conv.txt', 'seed2015_conv.txt', 
#                'seed2016_conv.txt', 'seed2017_conv.txt', 'seed2018_conv.txt', 'seed2019_conv.txt', 'seed2020_conv.txt', 'seed2021_conv.txt', 
#                'seed2022_conv.txt']

datasetsDef = ['seed2010_conv.txt']

person = ['edad_c', 'edad', 'nom_c', 'sexo', 'curp']
fechas = ['nac_dia', 'nac_mes', 'nac_anio', 'def_dia', 'def_mes', 'def_anio']
dir = ['res_ent', 'res_mun', 'res_loc', 'def_ent', 'def_mun', 'def_loc', 'causa_bas']
#falta = ['res_tipo_asen', 'res_nom_asen', 'def_tipo_asen', 'def_nom_asen']
infoIdenti = pd.DataFrame()

for i in datasetsDef:
    bd = pd.read_csv('dgis_datos/defuncion/' + i, delimiter='|', parse_dates=False, encoding='utf-8')
    bd = bd[person + fechas + dir]
    infoIdenti = pd.concat([infoIdenti, bd])
    #infoIdenti.append(bd)
infoIdenti

C:\Users\Hapy_\AppData\Local\Temp\ipykernel_15904\3763495792.py:14: DtypeWarning: Columns (32,33,38,39,44,76,107,108,116) have mixed types. Specify dtype option on import or set low_memory=False.
  bd = pd.read_csv('dgis_datos/defuncion/' + i, delimiter='|', parse_dates=False, encoding='utf-8')


,edad_c,edad,nom_c,sexo,curp,nac_dia,nac_mes,nac_anio,def_dia,def_mes,def_anio,res_ent,res_mun,res_loc,def_ent,def_mun,def_loc,causa_bas
0,0,999,MENDEZ GARCIA ROMAN,1,NaN,99,99,9999,26,1,2010,2,2001,20010001,2,2001,20010001,Y049
1,0,999,DESCONOCIDO DESCONOCIDO DESCONOCIDO,1,NaN,99,99,9999,12,10,2010,99,99999,999999999,15,15099,150990001,X997
2,0,999,DESCONOCIDO DESCONOCIDO DESCONOCIDO,1,NaN,99,99,9999,11,6,2010,99,99999,999999999,28,28041,280410001,X954
3,0,999,DESCONOCIDO DESCONOCIDO DESCONOCIDO,0,NaN,99,99,9999,13,3,2010,99,99999,999999999,15,15013,150130001,Y244
4,0,999,DESCONOCIDO DESCONOCIDO DESCONOCIDO,0,NaN,99,99,9999,26,4,2010,99,99999,999999999,15,15060,150600001,W849
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
560190,9,999,DESCONOCIDO DESCONOCIDO DESCONOCIDO,1,NaN,99,99,9999,17,9,2010,99,99999,999999999,26,26043,260430001,X444
560191,9,999,JIMENEZ MORALES GREGORIO,1,NaN,99,99,9999,2,10,2010,99,99999,999999999,99,99999,999999999,Y094
560192,9,999,DESCONOCIDO DESCONOCIDO DESCONOCIDO,0,NaN,99,99,9999,12,9,2010,99,99999,999999999,7,7035,70350001,W348
560193,9,999,DESCONOCIDO DESCONOCIDO DESCONOCIDO,1,NaN,99,99,9999,2,3,2010,99,99999,999999999,26,26043,260430001,X919


In [4]:
##########################################Edad##########################################

##########################edad_c##########################
#0: No especificado
#9: Se ignora
faltante = len(infoIdenti[((infoIdenti.edad_c >= 0) & (infoIdenti.edad_c <= 5)) | (infoIdenti.edad_c == 9)])
print('Datos faltantes: ', len(infoIdenti) - faltante)
noEsp = len(infoIdenti[(infoIdenti.edad_c == 0) | (infoIdenti.edad_c == 9)])
print('Edad no especificada: ', noEsp)

menos1An = len(infoIdenti[(infoIdenti.edad_c > 0) & (infoIdenti.edad_c < 5)])
print('Menores a 1 año: ', menos1An)
mas1An = len(infoIdenti[(infoIdenti.edad_c == 5)])
print('Mayores a 1 año: ', mas1An)
##########################edad##########################
#Edades de 0 a 130 años
#999: Desconocido 
#003: Minutos
faltante = len(infoIdenti[((infoIdenti.edad >= 0) & (infoIdenti.edad <= 130)) | (infoIdenti.edad <= 999)])
print('\n\nDatos faltantes: ', len(infoIdenti) - faltante)
noEsp = len(infoIdenti[(infoIdenti.edad == 999)])
print('Edad no especificada: ', noEsp)
recNac = len(infoIdenti[(infoIdenti.edad_c > 0) & (infoIdenti.edad_c < 3)])
print('Recien nacidos', recNac)

infoIdenti[(infoIdenti.edad_c > 0) & (infoIdenti.edad_c < 3)]

Datos faltantes:  0
Edad no especificada:  486
Menores a 1 año:  26983
Mayores a 1 año:  532726


Datos faltantes:  0
Edad no especificada:  486
Recien nacidos 5386


,edad_c,edad,nom_c,sexo,curp,nac_dia,nac_mes,nac_anio,def_dia,def_mes,def_anio,res_ent,res_mun,res_loc,def_ent,def_mun,def_loc,causa_bas
380,2,0,ALVARDO CABRERA LUIS,1,NaN,27,4,2010,27,4,2010,21,21159,211590002,21,21159,211590002,P073
381,2,0,ROJAS ALDAY RN,1,NaN,9,11,2010,9,11,2010,10,10030,100300001,10,10007,100070001,P072
382,2,0,GALLARDO MERCADO DALIA WENDOLI,2,NaN,17,4,2010,17,4,2010,19,19026,190260001,19,19039,190390001,Q913
383,2,0,PRIEGO CORDOVA RECIEN NACIDO,2,NaN,29,8,2010,29,8,2010,29,29010,290100001,29,29010,290100001,P011
384,2,0,GAMON GONZALEZ RN,2,NaN,31,10,2010,31,10,2010,8,8037,80370001,8,8037,80370001,P073
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5761,2,23,NaN,2,NaN,29,6,2010,29,6,2010,30,30072,300720001,13,13048,130480001,P220
5762,2,23,LUGO SANDOVAL RN,2,NaN,24,11,2010,25,11,2010,6,6010,60100001,6,6002,60020001,P369
5763,2,23,ONOFRE MARCIAL GUADALUPE,2,NaN,7,10,2010,8,10,2010,15,15088,150880043,15,15088,150880001,P220
5764,2,23,ARVIZU TRENADO RN,1,NaN,11,8,2010,12,8,2010,22,22014,220140001,22,22014,220140001,Q790


In [5]:
##########################################nom_c##########################################

In [6]:
##########################################sexo##########################################
#0: No especificado
#9: Se ignora
sexo_falt = len(infoIdenti[((infoIdenti.sexo >= 0) & (infoIdenti.sexo <= 2)) | (infoIdenti.sexo == 9)])
print('Datos faltantes: ', len(infoIdenti) - sexo_falt)
sexo_NoEp = len(infoIdenti[(infoIdenti.sexo == 9)])
print('Sexo desconocido: ', sexo_NoEp)
sexo_Hom = len(infoIdenti[(infoIdenti.sexo == 1)])
print('Hombres: ', sexo_Hom)
sexo_Muj = len(infoIdenti[(infoIdenti.sexo == 2)])
print('Mujeres: ', sexo_Muj)


Datos faltantes:  0
Sexo desconocido:  110
Hombres:  311823
Mujeres:  248136


In [7]:
##########################################curp##########################################

curp_falt = len(infoIdenti[(pd.isna(infoIdenti['curp']))])
print('Datos faltantes: ', curp_falt)

sinCURP = infoIdenti[(pd.isna(infoIdenti['curp']) == False)]
conCURP = sinCURP[(sinCURP['curp'].str.contains('^[A-Z]{4}[0-9]{2}(0[1-9]|1[0-2])(0[1-9]|1[0-9]|2[0-9]|3[0-1])[A-Z]{6}[0-9A-Z]{1}[0-9]{1}', case=False))]
total = len(sinCURP) - len(conCURP)
print('Curp invalida: ', total)
print('Curp Valida: ', len(infoIdenti) - (total + curp_falt))
len(infoIdenti)
sinCURP[(sinCURP['curp'].str.contains('^[A-Z]{4}[0-9]{2}(0[1-9]|1[0-2])(0[1-9]|1[0-9]|2[0-9]|3[0-1])[A-Z]{6}[0-9A-Z]{1}[0-9]{1}', case=False) == False)]

Datos faltantes:  513303
Curp invalida:  1197
Curp Valida:  45695


C:\Users\Hapy_\AppData\Local\Temp\ipykernel_15904\2567963265.py:7: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  conCURP = sinCURP[(sinCURP['curp'].str.contains('^[A-Z]{4}[0-9]{2}(0[1-9]|1[0-2])(0[1-9]|1[0-9]|2[0-9]|3[0-1])[A-Z]{6}[0-9A-Z]{1}[0-9]{1}', case=False))]


560195

C:\Users\Hapy_\AppData\Local\Temp\ipykernel_15904\2567963265.py:12: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  sinCURP[(sinCURP['curp'].str.contains('^[A-Z]{4}[0-9]{2}(0[1-9]|1[0-2])(0[1-9]|1[0-9]|2[0-9]|3[0-1])[A-Z]{6}[0-9A-Z]{1}[0-9]{1}', case=False) == False)]


,edad_c,edad,nom_c,sexo,curp,nac_dia,nac_mes,nac_anio,def_dia,def_mes,def_anio,res_ent,res_mun,res_loc,def_ent,def_mun,def_loc,causa_bas
2842,2,2,SALDAÑA SOLANO RN,2,LLLL999999LLLLLL99,3,9,2010,3,9,2010,12,12051,120510006,12,12051,120510001,Q897
3114,2,3,DIEGO DIRCIO NIÑO,1,LLLL999999LLLLLL99,6,7,2010,6,7,2010,12,12074,120740001,12,12074,120740001,P968
3490,2,5,SANCHEZ RIVAS RN,2,LLLL999999LLLLLL99,24,4,2010,24,4,2010,13,13021,130210006,13,13008,130080141,P285
5744,2,23,LOPEZ MARTINEZ RN,1,LLLL999999LLLLLL99,6,7,2010,7,7,2010,13,13007,130070001,13,13008,130080141,P220
6566,3,1,GONZALEZ MARCELO KAMILA,2,LLLL999999LLLLLL99,9,9,2010,10,9,2010,12,12029,120290001,12,12029,120290001,Q249
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
559726,5,107,VILLANUEVA CORTEZ JUANA,2,LLLL999999LLLLLL99,24,6,1902,11,2,2010,12,12028,120280031,12,12028,120280031,R54X
559811,5,107,GASPAR ANGELINO PEDRO SANTIAGO,1,LLLL999999LLLLLL99,18,9,1902,6,2,2010,12,12028,120280108,12,12028,120280108,R54X
559909,5,109,DE ATILANO BASTIAN MARIA PETRONILA,2,LLLL999999LLLLLL99,24,1,1901,18,3,2010,12,12028,120280108,12,12028,120280108,I219
559972,5,109,GUZMAN DE LA LUZ ANACLETO,1,LLLL999999LLLLLL99,20,11,1900,4,8,2010,12,12075,120750023,12,12075,120750023,E46X


In [8]:
##########################################Fecha Nacimiento##########################################
##########################nac_dia##########################
#99: Desconocido
nacDia_Des = len(infoIdenti[(infoIdenti.nac_dia == 99)])
nacDia_Valid = len(infoIdenti[(infoIdenti.nac_dia >= 1) & (infoIdenti.nac_dia <= 31)])
print('Dia de nacimiento faltante: ', len(infoIdenti) - (nacDia_Des + nacDia_Valid))
print('Dia de nacimiento desconocido: ', nacDia_Des)
print('Dia de nacimiento Valido: ', nacDia_Valid)

##########################nac_mes##########################
#99: Desconocido
nacMes_Des = len(infoIdenti[(infoIdenti.nac_mes == 99)])
nacMes_Valid = len(infoIdenti[(infoIdenti.nac_mes >= 1) & (infoIdenti.nac_mes <= 12)])
print('\n\nMes de nacimiento faltante: ', len(infoIdenti) - (nacMes_Des + nacMes_Valid))
print('Mes de nacimiento desconocido: ', nacMes_Des)
print('Mes de nacimiento Valido: ', nacMes_Valid)

##########################nac_anio##########################
#99: Desconocido
nacAnio_Des = len(infoIdenti[(infoIdenti.nac_anio == 9999)])
nacAnio_Invalid = len(infoIdenti[(pd.isna(infoIdenti['nac_anio']))])
print('\n\nAnio de nacimiento faltante: ', nacAnio_Invalid)
print('Anio de nacimiento desconocido: ', nacAnio_Des)
print('Anio de nacimiento Valido: ', len(infoIdenti) - (nacAnio_Invalid + nacAnio_Des))


Dia de nacimiento faltante:  0
Dia de nacimiento desconocido:  2739
Dia de nacimiento Valido:  557456


Mes de nacimiento faltante:  0
Mes de nacimiento desconocido:  2739
Mes de nacimiento Valido:  557456


Anio de nacimiento faltante:  0
Anio de nacimiento desconocido:  2745
Anio de nacimiento Valido:  557450


In [9]:
##########################################Fecha Nacimiento##########################################
##########################def_dia##########################
#99: Desconocido
nacDia_Des = len(infoIdenti[(infoIdenti.def_dia == 99)])
nacDia_Valid = len(infoIdenti[(infoIdenti.def_dia >= 1) & (infoIdenti.def_dia <= 31)])
print('Dia de fallecimiento faltante: ', len(infoIdenti) - (nacDia_Des + nacDia_Valid))
print('Dia de fallecimiento desconocido: ', nacDia_Des)
print('Dia de fallecimiento Valido: ', nacDia_Valid)

##########################def_mes##########################
#99: Desconocido
nacMes_Des = len(infoIdenti[(infoIdenti.def_mes == 99)])
nacMes_Valid = len(infoIdenti[(infoIdenti.def_mes >= 1) & (infoIdenti.def_mes <= 12)])
print('\n\nMes de fallecimiento faltante: ', len(infoIdenti) - (nacMes_Des + nacMes_Valid))
print('Mes de fallecimiento desconocido: ', nacMes_Des)
print('Mes de fallecimiento Valido: ', nacMes_Valid)

##########################def_anio##########################
#99: Desconocido
nacAnio_Des = len(infoIdenti[(infoIdenti.def_anio == 9999)])
nacAnio_Invalid = len(infoIdenti[(pd.isna(infoIdenti['def_anio']))])
print('\n\nAnio de fallecimiento faltante: ', nacAnio_Invalid)
print('Anio de fallecimiento desconocido: ', nacAnio_Des)
print('Anio de fallecimiento Valido: ', len(infoIdenti) - (nacAnio_Invalid + nacAnio_Des))

Dia de fallecimiento faltante:  0
Dia de fallecimiento desconocido:  72
Dia de fallecimiento Valido:  560123


Mes de fallecimiento faltante:  0
Mes de fallecimiento desconocido:  70
Mes de fallecimiento Valido:  560125


Anio de fallecimiento faltante:  0
Anio de fallecimiento desconocido:  67
Anio de fallecimiento Valido:  560128


In [21]:
##########################################Recidencia##########################################
#dir = ['res_ent', 'res_mun', 'res_loc', 'def_ent', 'def_mun', 'def_loc', 'causa_bas']
##########################res_ent##########################
#00: No especificado   33: EUA
#88: No aplica         34: Otros paises latinos  36: Mexico
#99: Se ignora         35: Otros paises
resEntEstad = len(infoIdenti[((infoIdenti.res_ent >= 1) & (infoIdenti.res_ent <= 32))])
resEntNoEp = len(infoIdenti[(infoIdenti.res_ent == 0) | (infoIdenti.res_ent == 99)])
resEntOtros = len(infoIdenti[((infoIdenti.res_ent >= 33) & (infoIdenti.res_ent <= 35)) | (infoIdenti.res_ent == 88)])
resEntEUM = len(infoIdenti[(infoIdenti.res_ent == 36)])
print('Entidad de residencia faltante: ', len(infoIdenti) - (resEntEstad + resEntNoEp + resEntOtros + resEntEUM))
print('Entidad de residencia no especificada: ', resEntNoEp)
print('Entidad de residencia: ', resEntEstad)
print('Entidad de residencia Mxico (sin esp estado): ', resEntEUM)
print('Entidad de residencia otros (USA, etc.): ', resEntOtros)
##########################res_mun##########################
#01998, 02998, ..., 32998: Se Ignora
#01999, 02999, ..., 32999: No Espec
#88997, 97997: No Aplica
#99998, 99999, 00999: No Espec 
#98998: Se Ignora
munNoEp = len(infoIdenti[((infoIdenti.res_mun%1000) == 999) | ((infoIdenti.res_mun%1000) == 998)])
munNoAp = len(infoIdenti[((infoIdenti.res_mun%1000) == 997)])
munVal = len(infoIdenti[((infoIdenti.res_mun >= 1001) & (infoIdenti.res_mun <= 32058)) & ((infoIdenti.res_mun%1000) != 997) & ((infoIdenti.res_mun%1000) != 998) & ((infoIdenti.res_mun%1000) != 999)])
print('\n\nMunicipios de residencia Datos extraños: ', len(infoIdenti) - (munNoEp + munNoAp + munVal))
print('Municipios de residencia desconocidos: ', munNoEp)
print('Municipios de residencia No aplica: ', munNoAp)
print('Municipios de residencia desconocidos: ', munVal)
temp = infoIdenti[((infoIdenti.res_mun >= 1001) & (infoIdenti.res_mun <= 32058) == False) & ((infoIdenti.res_mun%1000) != 997) & ((infoIdenti.res_mun%1000) != 998) & ((infoIdenti.res_mun%1000) != 999)]
temp
len(temp)
##########################res_loc##########################
#999999999: ?
#locNoEp = len(bd[(bd.res_loc == 999999999)])
#print('localidades de residencia desconocidos: ', locNoEp)

Entidad de residencia faltante:  0
Entidad de residencia no especificada:  1882
Entidad de residencia:  557554
Entidad de residencia Mxico (sin esp estado):  17
Entidad de residencia otros (USA, etc.):  742


Municipios de residencia Datos extraños:  37
Municipios de residencia desconocidos:  3621
Municipios de residencia No aplica:  0
Municipios de residencia desconocidos:  556537


,edad_c,edad,nom_c,sexo,curp,nac_dia,nac_mes,nac_anio,def_dia,def_mes,def_anio,res_ent,res_mun,res_loc,def_ent,def_mun,def_loc,causa_bas
19885,4,1,NaN,1,NaN,29,9,2010,8,11,2010,1,1000,10000000,9,9003,90030062,Q211
38238,5,14,NaN,1,NaN,7,6,1996,28,7,2010,36,36001,360010001,16,16045,160450021,X599
49748,5,20,RAMIREZ MARES RAFAEL,1,NaN,7,2,1990,4,7,2010,36,36001,360010001,16,16070,160700001,V892
50023,5,21,NaN,1,NaN,26,11,1988,14,4,2010,33,33000,330000000,2,2004,20040001,Y344
64190,5,26,JEAN MARIE KOLLER ETIENNE,1,NaN,4,12,1983,8,1,2010,35,35001,350010001,16,16066,160660001,V892
66919,5,28,HUAQIAO S/A XU,1,NaN,2,2,1982,13,7,2010,35,35001,350019999,31,31059,310590001,K922
76772,5,31,SANCHEZ JR RODOLFO,1,NaN,10,9,1978,2,2,2010,33,33001,330010001,16,16006,160060001,V99X
88969,5,35,CARRION BARRUETO LUIS EDUARDO,1,NaN,18,10,1975,30,12,2010,34,34001,340019999,31,31050,310500001,V892
93889,5,37,NaN,1,NaN,21,5,1973,17,8,2010,33,33001,330019999,31,31050,310500001,X599
127457,5,46,BARRAGAN LOPEZ MARTHA ALICIA,2,NaN,10,4,1963,27,2,2010,33,33001,330010001,16,16076,160760001,I219


37

In [22]:
##########################################Defuncion##########################################
#dir = ['res_ent', 'res_mun', 'res_loc', 'def_ent', 'def_mun', 'def_loc', 'causa_bas']
##########################def_ent##########################
#00: No especificado   33: EUA
#88: No aplica         34: Otros paises latinos  36: Mexico
#99: Se ignora         35: Otros paises
resEntEstad = len(infoIdenti[((infoIdenti.def_ent >= 1) & (infoIdenti.def_ent <= 32))])
resEntNoEp = len(infoIdenti[(infoIdenti.def_ent == 0) | (infoIdenti.def_ent == 99)])
resEntOtros = len(infoIdenti[((infoIdenti.def_ent >= 33) & (infoIdenti.def_ent <= 35)) | (infoIdenti.def_ent == 88)])
resEntEUM = len(infoIdenti[(infoIdenti.def_ent == 36)])
print('Entidad de defuncion faltante: ', len(infoIdenti) - (resEntEstad + resEntNoEp + resEntOtros + resEntEUM))
print('Entidad de defuncion no especificada: ', resEntNoEp)
print('Entidad de defuncion: ', resEntEstad)
print('Entidad de defuncion Mxico (sin esp estado): ', resEntEUM)
print('Entidad de defuncion otros (USA, etc.): ', resEntOtros)
##########################def_mun##########################
#01998, 02998, ..., 32998: Se Ignora
#01999, 02999, ..., 32999: No Espec
#88997, 97997: No Aplica
#99998, 99999, 00999: No Espec 
#98998: Se Ignora
munNoEp = len(infoIdenti[((infoIdenti.def_mun%1000) == 999) | ((infoIdenti.def_mun%1000) == 998)])
munNoAp = len(infoIdenti[((infoIdenti.def_mun%1000) == 997)])
munVal = len(infoIdenti[((infoIdenti.def_mun >= 1001) & (infoIdenti.def_mun <= 32058)) & ((infoIdenti.def_mun%1000) != 997) & ((infoIdenti.def_mun%1000) != 998) & ((infoIdenti.def_mun%1000) != 999)])
print('\n\nMunicipios de defuncion Datos extraños: ', len(infoIdenti) - (munNoEp + munNoAp + munVal))
print('Municipios de defuncion desconocidos: ', munNoEp)
print('Municipios de defuncion No aplica: ', munNoAp)
print('Municipios de defuncion validos: ', munVal)
temp = infoIdenti[((infoIdenti.def_mun >= 1001) & (infoIdenti.def_mun <= 32058) == False) & ((infoIdenti.def_mun%1000) != 997) & ((infoIdenti.def_mun%1000) != 998) & ((infoIdenti.def_mun%1000) != 999)]
temp
len(temp)
##########################res_loc##########################
#999999999: ?
#locNoEp = len(bd[(bd.res_loc == 999999999)])
#print('localidades de residencia desconocidos: ', locNoEp)

Entidad de defuncion faltante:  0
Entidad de defuncion no especificada:  836
Entidad de defuncion:  559350
Entidad de defuncion Mxico (sin esp estado):  0
Entidad de defuncion otros (USA, etc.):  9


Municipios de defuncion Datos extraños:  3
Municipios de defuncion desconocidos:  1026
Municipios de defuncion No aplica:  0
Municipios de defuncion desconocidos:  559166


,edad_c,edad,nom_c,sexo,curp,nac_dia,nac_mes,nac_anio,def_dia,def_mes,def_anio,res_ent,res_mun,res_loc,def_ent,def_mun,def_loc,causa_bas
42173,5,17,CEBREROS BUSTAMANTE BRIAN DA KARELI,2,NaN,16,10,1992,25,8,2010,25,25008,250080074,1,1000,10000000,X958
238811,5,63,NaN,1,NaN,23,6,1947,28,6,2010,35,35001,350019999,35,35001,350019999,C419
381210,5,78,BARRIOS OLIVARES OTTO ENRIQUE,1,NaN,16,9,1931,17,5,2010,34,34000,340000000,34,34000,340000000,I219


3